In [ ]:
#!/usr/bin/env python
# vim: set fileencoding=utf8 :
#```

#!pip install -U boto3 retrying
#!export AWS_DEFAULT_PROFILE=test


In [ ]:
!cat athena_taxi_raw.sql


In [ ]:
!python athena.py athena_taxi_raw.sql

In [3]:
!python athena.py athena_taxi_raw_2014.sql

In [4]:
!python athena.py athena_taxi_raw_2015.sql

In [ ]:
!ls -1

In [2]:
!cat athena.log

INFO:root:{'QueryExecutionId': '54e99fc2-8fef-4995-bdea-ef46fb362036',
 'ResponseMetadata': {'HTTPHeaders': {'connection': 'keep-alive',
                                      'content-length': '59',
                                      'content-type': 'application/x-amz-json-1.1',
                                      'date': 'Mon, 10 Jun 2019 09:38:43 GMT',
                                      'x-amzn-requestid': '37c76f81-e911-4ed5-a345-c4bb9e40a36c'},
                      'HTTPStatusCode': 200,
                      'RequestId': '37c76f81-e911-4ed5-a345-c4bb9e40a36c',
                      'RetryAttempts': 0}}
INFO:root:{'Query': 'WITH \n'
          '    dataset AS \n'
          '    (SELECT CAST (pickup_datetime AS TIMESTAMP WITH time zone) AT '
          "TIME ZONE 'America/New_York' AS est, \n"
          '                  ST_POINT(pickup_longitude,pickup_latitude) '
          'pickup_point,\n'
          '                  ST_POINT(dropoff_longitude,dropoff_latitude) '
       

athena.log  # program log
athena.py   # main program
foo.sql     # query execution result
foo.sql.csv # sql output